# Un ORM

Il più diffuso è [SQLAlchemy](https://docs.sqlalchemy.org/)

In [1]:
from sqlalchemy import create_engine

engine = create_engine('sqlite:///:memory:', echo = True)

In [2]:
from sqlalchemy.ext.declarative import declarative_base

Base = declarative_base()

In [3]:
from sqlalchemy import Column, Integer, String, Sequence

class User(Base):
    __tablename__ = 'user'

    id = Column(Integer, Sequence('user_id_seq'), primary_key = True)
    username = Column(String)
    password = Column(String)
    
    def __repr__(self):
        return f'<User: {self.username}>'

In [4]:
Base.metadata.create_all(engine)

2019-10-28 22:36:59,997 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2019-10-28 22:36:59,998 INFO sqlalchemy.engine.base.Engine ()
2019-10-28 22:37:00,000 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2019-10-28 22:37:00,001 INFO sqlalchemy.engine.base.Engine ()
2019-10-28 22:37:00,003 INFO sqlalchemy.engine.base.Engine PRAGMA main.table_info("user")
2019-10-28 22:37:00,004 INFO sqlalchemy.engine.base.Engine ()
2019-10-28 22:37:00,005 INFO sqlalchemy.engine.base.Engine PRAGMA temp.table_info("user")
2019-10-28 22:37:00,006 INFO sqlalchemy.engine.base.Engine ()
2019-10-28 22:37:00,008 INFO sqlalchemy.engine.base.Engine 
CREATE TABLE user (
	id INTEGER NOT NULL, 
	username VARCHAR, 
	password VARCHAR, 
	PRIMARY KEY (id)
)


2019-10-28 22:37:00,009 INFO sqlalchemy.engine.base.Engine ()
2019-10-28 22:37:00,011 INFO sqlalchemy.engine.base.Engine COMMIT


In [5]:
example_user = User()

In [6]:
example_user.username = 'Mapio'
example_user.password = 'mypw'

example_user

<User: Mapio>

In [7]:
from sqlalchemy.orm import sessionmaker

Session = sessionmaker(bind = engine)
session =  Session()

In [8]:
session.add(example_user)

In [9]:
session.add_all([
    User(username = 'Mickey', password = 'm0us3'),
    User(username = 'Donald', password = 'dukk')
])

In [10]:
session.commit()

2019-10-28 22:37:00,094 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2019-10-28 22:37:00,097 INFO sqlalchemy.engine.base.Engine INSERT INTO user (username, password) VALUES (?, ?)
2019-10-28 22:37:00,101 INFO sqlalchemy.engine.base.Engine ('Mapio', 'mypw')
2019-10-28 22:37:00,103 INFO sqlalchemy.engine.base.Engine INSERT INTO user (username, password) VALUES (?, ?)
2019-10-28 22:37:00,105 INFO sqlalchemy.engine.base.Engine ('Mickey', 'm0us3')
2019-10-28 22:37:00,107 INFO sqlalchemy.engine.base.Engine INSERT INTO user (username, password) VALUES (?, ?)
2019-10-28 22:37:00,109 INFO sqlalchemy.engine.base.Engine ('Donald', 'dukk')
2019-10-28 22:37:00,111 INFO sqlalchemy.engine.base.Engine COMMIT


In [11]:
session.query(User).order_by(User.username).all()

2019-10-28 22:37:00,122 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2019-10-28 22:37:00,125 INFO sqlalchemy.engine.base.Engine SELECT user.id AS user_id, user.username AS user_username, user.password AS user_password 
FROM user ORDER BY user.username
2019-10-28 22:37:00,128 INFO sqlalchemy.engine.base.Engine ()


[<User: Donald>, <User: Mapio>, <User: Mickey>]

In [12]:
session.query(User).filter_by(username = 'Donald').first()

2019-10-28 22:37:00,143 INFO sqlalchemy.engine.base.Engine SELECT user.id AS user_id, user.username AS user_username, user.password AS user_password 
FROM user 
WHERE user.username = ?
 LIMIT ? OFFSET ?
2019-10-28 22:37:00,145 INFO sqlalchemy.engine.base.Engine ('Donald', 1, 0)


<User: Donald>

In [13]:
from sqlalchemy import Text, DateTime, ForeignKey
from sqlalchemy.orm import relationship
from sqlalchemy.sql import func

class Post(Base):
    __tablename__ = 'post'

    id = Column(Integer, Sequence('user_id_seq'), primary_key = True)
    title = Column(String)
    created = Column(DateTime, server_default = func.now())
    body = Column(Text)

    author_id = Column(Integer, ForeignKey('user.id'))
    user = relationship("User", back_populates = 'posts')

    def __repr__(self):
        return f'<Post: {self.title}>'
    
User.posts = relationship('Post', order_by = Post.id, back_populates = 'user')

In [14]:
Base.metadata.create_all(engine)

2019-10-28 22:37:00,173 INFO sqlalchemy.engine.base.Engine PRAGMA main.table_info("user")
2019-10-28 22:37:00,175 INFO sqlalchemy.engine.base.Engine ()
2019-10-28 22:37:00,177 INFO sqlalchemy.engine.base.Engine PRAGMA main.table_info("post")
2019-10-28 22:37:00,177 INFO sqlalchemy.engine.base.Engine ()
2019-10-28 22:37:00,179 INFO sqlalchemy.engine.base.Engine PRAGMA temp.table_info("post")
2019-10-28 22:37:00,181 INFO sqlalchemy.engine.base.Engine ()
2019-10-28 22:37:00,182 INFO sqlalchemy.engine.base.Engine 
CREATE TABLE post (
	id INTEGER NOT NULL, 
	title VARCHAR, 
	created DATETIME DEFAULT (CURRENT_TIMESTAMP), 
	body TEXT, 
	author_id INTEGER, 
	PRIMARY KEY (id), 
	FOREIGN KEY(author_id) REFERENCES user (id)
)


2019-10-28 22:37:00,184 INFO sqlalchemy.engine.base.Engine ()
2019-10-28 22:37:00,185 INFO sqlalchemy.engine.base.Engine COMMIT


In [ ]:
CREATE TABLE user (
  id INTEGER PRIMARY KEY AUTOINCREMENT,
  username TEXT UNIQUE NOT NULL,
  password TEXT NOT NULL
);

CREATE TABLE post (
  id INTEGER PRIMARY KEY AUTOINCREMENT,
  author_id INTEGER NOT NULL,
  created TIMESTAMP NOT NULL DEFAULT CURRENT_TIMESTAMP,
  title TEXT NOT NULL,
  body TEXT NOT NULL,
  FOREIGN KEY (author_id) REFERENCES user (id)
);


In [15]:
example_post = Post()

In [16]:
example_post.title = 'This is new!'
example_post.body = """\
Now this is a very
interesting story…
"""
example_post.user = example_user

In [17]:
session.add(example_post)
session.commit()

2019-10-28 22:38:08,956 INFO sqlalchemy.engine.base.Engine INSERT INTO post (title, body, author_id) VALUES (?, ?, ?)
2019-10-28 22:38:08,957 INFO sqlalchemy.engine.base.Engine ('This is new!', 'Now this is a very\ninteresting story…\n', 1)
2019-10-28 22:38:08,960 INFO sqlalchemy.engine.base.Engine COMMIT


In [18]:
example_user.posts

[<Post: This is new!>]

In [30]:
list(engine.execute('select * from user'))
list(engine.execute('select * from post'))

2019-10-28 22:46:57,776 INFO sqlalchemy.engine.base.Engine select * from user
2019-10-28 22:46:57,777 INFO sqlalchemy.engine.base.Engine ()
2019-10-28 22:46:57,778 INFO sqlalchemy.engine.base.Engine select * from post
2019-10-28 22:46:57,779 INFO sqlalchemy.engine.base.Engine ()


[(1, 'This is new!', '2019-10-28 21:38:08', 'Now this is a very\ninteresting story…\n', 1)]